# Regresión Lineal Simple. Un ejemplo minimalista

### Importar las librerías relevantes

In [277]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para graficar en 3-D

### Generar datos al azar para entrenar al modelo

Se trabajará con dos variables de entrada, las x1 y x2 en los ejemplos vistos en clase. Se generan al azar a partir de una distribución uniforme.

Se creará una matriz con estas dos variables.  La matriz X del modelo lineal y = x * w + b

In [278]:
# Por facilidad, se declara una variable para indicar el tamaño del conjunto 
#      de datos de entrenamiento. Puede probarse con 100,000 o 1,000,000 pero hay 
#      que tener cuidado ya que con tantas observaciones puede que la máquina se quede!

observaciones = 1000

x1 = np.random.uniform(low = -10, high = 10, size = (observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

X = np.column_stack((x1,x2))

# Verificar la forma de la matriz 
# Debiera ser n x k, donde n es el número de observaciones, y k es el número de variables.

print (X.shape)

(1000, 2)


### Generar las metas a las que debemos apuntar

Para el modelo se usará la función f(x1, x2) = 2 * x1 - 3 * x2 + 5 + <ruido pequeño>.  El ruido es para hacerlo más realista.

Se utiliza la metodología de ML, y al finalizar se determinará si el algoritmo la ha aprendido.  

Al utilizar esta función, se espera que la red neuronal genere los pesos w1 = 2, w2 = -3 y el sesgo b = 5.  Si no se logra, es que algo se ha hecho mal. 

In [279]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

metas = 13 * x1 + 7 * x2 - 12 + ruido

# Verificar las dimensiones. Deben ser n x m, donde m es el número de variables de salida.
print (metas.shape)

(1000, 1)


### Graficar los datos a usar para el entrenamiento

La idea es ver que haya una fuerte tendencia que el modelo debe aprender a reproducir.


In [280]:
print(x1.shape)
print(x2.shape)
print(metas.shape)

(1000, 1)
(1000, 1)
(1000, 1)


In [281]:
x1N = x1.reshape(observaciones,)
x2N = x2.reshape(observaciones,)
metasN = metas.reshape(observaciones,)

fig = px.scatter_3d(x = x1N, y = x2N, z = metasN)

fig.update_layout(
    width = 500,
    height = 500,)

fig.show()

### Inicializar variables

Se inicializan los pesos y sesgos, al azar, dentro de un rango inicial pequeño.  Es posible "jugar" con este valor pero no es recomendable ya que el uso de rangos iniciales altos inhibe el aprendizaje por parte del algoritmo

Los pesos son de dimensiones k x m, donde k es el numero de variables de entrada y m es el número de variables de salida.  

Como solo hay una salida, el sesgo es de tamaño 1, y es un escalar

In [282]:
rango_inicial = 0.1     #  valor máximo para los pesos y sesgos iniciales

pesos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size = (2, 1))

sesgos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size = 1)

#Ver cómo fueron inicializados.
print (pesos)
print (sesgos)

[[-0.03007089]
 [-0.08075232]]
[-0.0950443]


In [283]:
pesos.shape

(2, 1)

### Asignar la tasa de aprendizaje (Eta)

Se asigna una tasa de aprendizaje pequeña.  Para este ejemplo funciona bien 0.02.  Vale la pena "jugar" con este valor para ver los resultados de hacerlo.

In [284]:
eta = 0.02

### Entrenar el modelo

Se utilizará un valor de 100 para iterar el modelo con el conjunto de datos de entrenamiento.  Ese valor funciona bastante bien con la tasa de aprendizaje de 0.02.  Cómo saber el número adecuado de iteraciones es algo que se verá en futuras sesiones, pero generalmente una tasa de aprendizaje baja requiere de más iteraciones que una más alta.  Sin embargo hay que tener en mente que una tasa de aprendizaje alta puede causar que la pérdida "Loss" diverja a infinito, en vez de converger a cero (0)

Puesto que esta es una regresión, se usará la función de pérdida L2-norm (dividido por 2, para ser consistente con la clase).  Es más, también se dividirá por el número de observaciones para obtener un promedio de pérdida por observación.  Se discutió en clase sobre la libertad de modificar esta función una vez no se pierda la característica de ser más baja para los resultados mejores, y vice versa.

Se mostrará la función de pérdida (loss) en cada iteración, para ver si está decreciendo como se desea.

Otro pequeño truco es escalar las deltas de la misma manera que se hizo con la función de pérdida.  De esta forma la tasa de aprendizaje es independiente del número de observaciones.  De nuevo esto no cambia el principio, solo hace más fácil la selección de una tasa única de aprendizaje. 

Finalmente se aplica la regla de actualización del decenso de gradiente.

Ojo!  los pesos son de dimensión 2 X 1, la tasa de aprendizaje es 1 X 1 (escalar), las entradas son 1000 X 2, y las deltas escaladas son 1000 X 1.  Es necesario obtener la transpuesta de las entradas para que no hayan problemas de dimensión en las operaciones. 



In [285]:
for i in range (10000):
    
    # Esta es la ecuacion del modelo lineal: y = xw + b 
    y = np.dot(X, pesos) + sesgos
    
    # Las deltas son las diferencias entre las salidas y las metas (targets)
    # deltas es un vector 1000 x 1
    deltas = y - metas
        
    # Cambio a pérdida de L1-norm
    perdida = np.sum(np.abs(deltas)) / observaciones
    
    print(perdida)
    
    deltas_escaladas = deltas / observaciones
      
    pesos = pesos - eta * np.dot(X.T, deltas_escaladas)
    sesgos = sesgos - eta * np.sum(deltas_escaladas)


72.91820204004557
24.736141214559414
12.157688838338121
11.169432171017595
10.99850785407364
10.796038881137534
10.586075539118761
10.376486321878794
10.169815501463017
9.966850949401138
9.767800078699311
9.572678604385622
9.381439446719334
9.194015589198324
9.010334357577532
8.830322177966199
8.653906153787117
8.481014576966677
8.311577081287986
8.145524675513162
7.982789736245653
7.823305987517626
7.667008476227387
7.513833546515765
7.363718814124803
7.216603141087655
7.072426610862123
6.93113050393962
6.792657273934205
6.6569505241466835
6.523954984596147
6.39361648951005
6.265881955263859
6.1406993587614895
6.018017716247555
5.897787062542971
5.779958430695382
5.664483832036154
5.551316236635747
5.44040955414965
5.331718615046873
5.225199152213574
5.120807782924118
5.018501991172411
4.918240110356175
4.819981306307166
4.723685560660451
4.629313654555922
4.536827152665418
4.446188387538983
4.357360444263857
4.2703071454299355
4.184993036395615
4.101383370848016
4.0194440966516565
3.

0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723
0.49219562685896723


### Desplegar los pesos y el sesgo para ver si funcionaron correctamente.

Por el diseño de los datos, los pesos finales deben ser 2 y -3, y el sesgo: 5

**NOTA:**  Si aún no están los valores correctos, puede que aún estén convergiendo y sea necesario iterar más veces.  Para esto solo se requiere ejecutar la celda anterior cuantas veces sea requerido

In [286]:
print(pesos, sesgos)      

[[12.9974838]
 [ 6.9995651]] [-11.9927164]


### Graficar las últimas salidas vrs las metas (targets)

Como son los últimos valores, luego del entrenamiento, representan la exactitut del modelo final de.  Entre más cercana esté esta gráfica a una línea de 45 grados, más cercanas están las salidas y metas.

Como este ejemplo es pequeño, es posible hacerlo, en los problemas que se veran posteriormente en el curso, esto ya no sería posible.

In [287]:
yN = y.reshape(observaciones,)
metasN = metas.reshape(observaciones,)
fig = px.scatter(x = yN, y =  metasN)

fig.update_layout(
    width = 400,
    height = 400,)

fig.show()